In [8]:
%matplotlib inline

import sys
import random
import pickle
import chromadb
import numpy as np
from pprint import pprint
from tqdm.auto import tqdm
from matplotlib import pyplot as plt
import networkx as nx
import json
from pyvis.network import Network

sys.path.append("../")
from utils.vector_search_utils import *
from utils.parse_arxiv import *
from compute_metrics import batch_compute_percent_include

chroma_client = chromadb.PersistentClient(path="../data/chroma_dbs/")
chroma_client.list_collections()

[Collection(name=cnn_headline),
 Collection(name=arxiv_title),
 Collection(name=arxiv_abstract),
 Collection(name=wiki_plot),
 Collection(name=wiki_plot_summary),
 Collection(name=cnn_article)]

In [11]:
test_coll = chroma_client.get_collection("arxiv_title")
gt_coll = chroma_client.get_collection("arxiv_abstract")

# read workload csv
workload_csv = "../data/arxiv/arxiv_workloads/pn1000_n1.csv"
workload_df = pd.read_csv(workload_csv)

text_queries = workload_df["query"].tolist()

In [16]:
k = 10
test_k = k 

gt_res = vector_search(coll=gt_coll, text_queries=text_queries, k=k, batch_size=100)
test_res = vector_search(coll=test_coll, text_queries=text_queries, k=test_k, batch_size=100)

100%|██████████| 10/10 [00:10<00:00,  1.06s/it]


In [23]:
def exp(gt_coll, test_coll, k, kps, text_queries, gt_res, test_res):
    mean_recalls = []
    gt_res = vector_search(coll=gt_coll, text_queries=text_queries, k=k, batch_size=100)
    for kp in kps:
        # increase test k
        test_k = int( (1 + kp) * k)
        test_res = vector_search(coll=test_coll, text_queries=text_queries, k=test_k, batch_size=100)
        recalls = batch_compute_percent_include(gt_res, test_res)
        mean_recalls.append(np.mean(recalls))
    return mean_recalls

In [29]:
# make plot mean-recalls vs. % of extra k (0 to 100% increase)
ks = [5, 10, 50, 100, 500]
kps = np.arange(0, 1.2, 0.2)

mean_recall_dict = {}
for k in ks:
    mean_recalls = exp(gt_coll, test_coll, k, kps, text_queries, gt_res, test_res)
    mean_recall_dict[k] = mean_recalls

 50%|█████     | 5/10 [00:05<00:05,  1.05s/it]

In [28]:
# plot
# plt.plot(kps, mean_recalls)

array([0. , 0.2, 0.4, 0.6, 0.8, 1. ])

In [26]:
mean_recalls

[0.254,
 0.254,
 0.2786,
 0.2786,
 0.302,
 0.302,
 0.323,
 0.323,
 0.34020000000000006,
 0.34020000000000006,
 0.355]

# create_random_adjacency_matrix

In [ ]:
# create a random adjacency matrix (graph), user can also specify their numbers of random edges
def create_random_adjacency_matrix(n, num_edges=None):
    adj = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            if random.random() < 0.5:
                adj[i, j] = 1
                adj[j, i] = 1
    if num_edges is not None:
        edges = np.argwhere(adj == 1)
        if len(edges) > num_edges:
            edges = random.sample(list(edges), num_edges)
            adj = np.zeros((n, n))
            for i, j in edges:
                adj[i, j] = 1
                adj[j, i] = 1
    return adj

# a func take in an adjacency matrix and return eigenvalues and eigenvectors
def get_eigen(adj):
    eigenvalues, eigenvectors = np.linalg.eig(adj)
    return eigenvalues, eigenvectors

In [ ]:
g = create_random_adjacency_matrix(10, 10)
# print(g)
# print()
eigenvalues, eigenvectors = get_eigen(g)
# print(eigenvalues.sum())
# print(eigenvectors)

# make a plot of eigenvalues vs. number of edges (0 to 100, step 10)
n = 100
num_edges = []
eigenvalues_sum = []
for ne in range(0, 1000, 20):
    num_edges.append(ne)
    # print(num_edges)
    g = create_random_adjacency_matrix(n, ne)
    eigenvalues, eigenvectors = get_eigen(g)
    eigenvalues_sum.append(eigenvalues.max())
    # print(eigenvalues.sum())

plt.plot(num_edges, eigenvalues_sum)
plt.xlabel('Number of edges')
plt.ylabel('MAX Eigenvalues')
plt.title('Eigenvalues vs. number of edges (graph size = {}x{})'.format(n,n))
plt.show()

In [ ]:
 # load pickle
g_data = pickle.load(open('../data/arxiv/graph.pickle', 'rb'))
g_data.data_dict['id_1009.5145']

In [ ]:
G = nx.Graph()
G.add_nodes_from([(2, {"color": "red"}), (3, {"color": "blue"}), (4, {"color": "green"})])
colors = [G.nodes[n]['color'] for n in G.nodes]
print(colors)

# nx.draw(G, with_labels=True, node_color=colors)

In [ ]:
# G.keyword_dict.keys()

# a func take in a list and return all combinations of 2 elements, do not use list comprehension
def get_pair_combinations(l):
    result = []
    for i in range(len(l)):
        try: 
            result.append((l[i], l[i+1]))
        except:
            result.append((l[i], l[0]))
        # for j in range(i+1, len(l)):
        #     result.append((l[i], l[j]))
    return result

In [ ]:
class MetadataGraph:
    def __init__(self):
        self.G = nx.Graph()
        self.metadata_nodes = []

    def update_one(self, metadata_set):
        self.add_data_nodes(metadata_set)
        self.connect_edges(metadata_set)

    def add_data_nodes(self, metadata_set):
        metadata_list = list(metadata_set)
        # color hex code random generator
        random_color = "#{:06x}".format(random.randint(0, 0xFFFFFF))

        # construct a list of tuples, each tuple is a node and its attributes: (node, {attr1: val1, attr2: val2, ...})
        metadata_nodes_with_attrs = []
        for md in metadata_list:
            metadata_nodes_with_attrs.append((md, {"color": random_color}))

        self.G.add_nodes_from(metadata_nodes_with_attrs)
        self.metadata_nodes.extend(metadata_list)

    def connect_edges(self, metadata_set):
        metadata_list = list(metadata_set)
        edge_list = get_pair_combinations(metadata_list)
        self.G.add_edges_from(edge_list)

    def get_adjacency_matrix(self):
        return nx.adjacency_matrix(self.G).todense()
    
    def get_laplacian_matrix(self, normalized=False):
        if normalized:
            return nx.normalized_laplacian_matrix(self.G).todense()
        
        return nx.laplacian_matrix(self.G).todense()
    
    def draw(self, with_labels=False, node_size=15, alpha=0.85, subplot_ax=None, color_scheme=None):
        if color_scheme==None:
            colors = [self.G.nodes[n]['color'] for n in self.G.nodes]
        else:
            colors = color_scheme
        nx.draw(self.G, with_labels=with_labels, node_color=colors, node_size=node_size, pos=nx.spring_layout(self.G), alpha=alpha, ax=subplot_ax)

# Color scheme

- color by metadata types

In [ ]:
def get_cat_from_keyword(keyword):
    return keyword.split(':')[0]

def color_scheme_gen(num):
    '''
    Generate a list of random color hex codes, each code is a string. 
    '''
    color_list = []
    for i in range(num):
        color_list.append("#{:06x}".format(random.randint(0, 0xFFFFFF)))
    return color_list

def plot_color_scheme(color_list, text_list=None):
    '''
    Plot the color scheme, each color is a hex code string. horizontal color bars with text after each bar.

    Args:
        - color_list: list of color hex codes
        - text_list: list of text strings, each string is the name of the color
    '''
    height = 0.9
    width = 0.6

    fig, ax = plt.subplots(figsize=(3.5, 2.5))
    for i, c in enumerate(color_list):
        ax.add_patch(plt.Rectangle((0, i), width, height, color=c))
    
    if text_list is not None:
        for i, t in enumerate(text_list):
            ax.text(width*1.02, i+0.5*height, t, ha='left', va='center', fontsize=9, color='black')
        
    ax.set_ylim(0, len(color_list))
    # ax.set_xlim(0, 1)
    ax.axis('off')
    plt.tight_layout()
    plt.show()


arxiv_meta_cats = ['author', 'journal', 'category', 'year']
num = len(arxiv_meta_cats)
color_list = color_scheme_gen(num)
color_dict = {m:c for m, c in zip(arxiv_meta_cats, color_list)}
plot_color_scheme(color_list, arxiv_meta_cats)

In [ ]:
# keys = random.sample(list(g_data.data_dict.keys()), 10)
# g_data.data_dict[keys[0]]

# color_dict

In [ ]:
# all_meta_cats = list(map(get_cat_from_keyword, list(mg.G.nodes)))
# all_meta_colors = list(map(color_dict.get, all_meta_cats))

In [ ]:
# all_meta_colors[:10]

In [ ]:
# random pick keys 
keys = random.sample(list(g_data.data_dict.keys()), 10)

# test the class
mg = MetadataGraph()

for k in keys:
    print("add {}".format(k))
    mg.update_one(g_data.data_dict[k])

    # meta color
    all_meta_cats = list(map(get_cat_from_keyword, list(mg.G.nodes)))
    all_meta_colors = list(map(color_dict.get, all_meta_cats))
    
    plt.figure(figsize=(4.5, 3))
    # mg.draw(color_scheme=all_meta_colors)
    mg.draw()
    plt.show()
    
nt = Network('700px', '900px', notebook=True)
nt.from_nx(mg.G)
nt.toggle_physics(True)
nt.show('nx.html')

In [ ]:
# pprint(mg.G.edges())

In [ ]:
# g_data.data_dict['id_2308.02657']

In [ ]:
# mg.G.nodes['category:cs.IT']

# ChromaDB

In [ ]:
# chroma_client = chromadb.PersistentClient(path="../data/chroma_dbs/")
# chroma_client.list_collections()

In [ ]:
G_data = pickle.load(open('../data/arxiv/graph.pickle', 'rb'))

# read json result file
results = json.load(open('../data/arxiv/arxiv_workloads/res/k100_gk20_pn50_n20.json', 'r'))
results['1'].keys()

In [ ]:
def remove_id_from_list(id_list, id_to_remove):
    return [id for id in id_list if id != id_to_remove]

def random_paper_id_swap(id_list, id_pool, num, from_end=True, random_=None):
    '''
    Randomly swap paper ids in the id_list with ids in the id_pool (not in the id_list) without replacement.

    args:
        - id_list: list of paper ids
        - id_pool: list of paper ids (not in the id_list)
        - num: number of ids to swap
        - from_end: if True, swap ids from the end of the id_list, else from the beginning
        - random: if True, randomly remove id from id_list, will OVERRIDE from_end args (in development)
    '''
    if num > len(id_pool) or num==0:
        return id_list

    # randomly pick num of ids from id_pool
    swap_ids = random.sample(id_pool, num)

    if random_ is None:
        if from_end:
            # remove num of ids from the end of id_list
            id_list = id_list[:-num]
            id_list.extend(swap_ids)
        else:
            # remove num of ids from the beginning of id_list
            id_list = id_list[num:]
            id_list = swap_ids + id_list
    else:
        # randomly remove num ids from id_list, and insert swap_ids
        raise NotImplementedError
    
    return id_list

In [ ]:
paper_id_pool = list(G_data.data_dict.keys())

In [ ]:
sample_res = results['0']['arxiv_abstract']

good_recall_mg = MetadataGraph()
for k in tqdm(sample_res):
    good_recall_mg.update_one(g_data.data_dict[k])

In [ ]:
plt.figure(figsize=(4.5, 3))
all_meta_cats = list(map(get_cat_from_keyword, list(good_recall_mg.G.nodes)))
all_meta_colors = list(map(color_dict.get, all_meta_cats))
# good_recall_mg.draw(node_size=13, alpha=0.75, color_scheme=all_meta_colors)
# plt.show()

# get eigenvalues for the laplacian matrix
eigenvalues, eigenvectors = get_eigen(good_recall_mg.get_laplacian_matrix(normalized=True))
print(eigenvalues.real.max())

nt = Network('700px', '900px', notebook=True)
nt.from_nx(good_recall_mg.G)
nt.toggle_physics(True)
# nt.show('nx.html')

In [ ]:
# nt = Network('1000px', '1000px', notebook=True)
# nt.from_nx(good_recall_mg.G)
# nt.toggle_physics(True)
# nt.show('nx.html')

In [ ]:
swapped_res = random_paper_id_swap(sample_res, remove_id_from_list(paper_id_pool, sample_res), 99, from_end=True)
bad_recall_mg = MetadataGraph()
for k in swapped_res:
    bad_recall_mg.update_one(g_data.data_dict[k])

bad_eigenvalues, bad_eigenvectors = get_eigen(bad_recall_mg.get_laplacian_matrix(normalized=True))

# plot good recall plot and bad recall plot side by side
fig, ax = plt.subplots(1, 2, figsize=(13, 5))
good_recall_mg.draw(node_size=10, alpha=0.7, subplot_ax=ax[0])
bad_recall_mg.draw(node_size=10, alpha=0.7, subplot_ax=ax[1])
ax[0].set_title('100% Recall (Eig = {:.3f})'.format(eigenvalues.real.max()))
ax[1].set_title('1% Recall (Eig = {:.3f})'.format(bad_eigenvalues.real.max()))
plt.show()

In [ ]:
recalls = []
numbers_of_edges = []
adjacency_eigenvalues = []
laplacian_eigenvalues = []

# given sample res
for num in tqdm(range(len(sample_res))):
    swapped_res = random_paper_id_swap(sample_res, remove_id_from_list(paper_id_pool, sample_res), num, from_end=True)
    mg = MetadataGraph()
    for k in swapped_res:
        mg.update_one(g_data.data_dict[k])

    numbers_of_edges.append(mg.G.number_of_edges())
    
    eigenvalues, _ = get_eigen(mg.get_adjacency_matrix())
    adjacency_eigenvalues.append(eigenvalues.real.max())

    eigenvalues, _ = get_eigen(mg.get_laplacian_matrix(normalized=True))
    laplacian_eigenvalues.append(eigenvalues.real.max())
    # eigenvalues = eigenvalues[eigenvalues > 0]
    # real_min_eigen = eigenvalues.real.min()
    # laplacian_eigenvalues.append(real_min_eigen)

    recalls.append( (len(sample_res) - num) / len(sample_res) )

In [ ]:
plt.figure(figsize=(4.5, 3))
mg.draw(node_size=13, alpha=0.75)
plt.show()

In [ ]:
# mg.get_laplacian_matrix(normalized=True)
# recalls

In [ ]:
# plot recall vs. number of edges
plt.plot(recalls, numbers_of_edges)
plt.xlabel('Recall')
plt.ylabel('Number of edges')
plt.title('Recall vs. number of edges')
plt.tight_layout()
plt.show()

In [ ]:
# plot
plt.plot(recalls, adjacency_eigenvalues)
plt.title('Recall vs. Max Eigenvalues (Adjacency Matrix)')
plt.xlabel('Recall')
plt.ylabel('Real Max Eigenvalues')
plt.tight_layout()
plt.show()

In [ ]:
plt.plot(recalls, laplacian_eigenvalues)
plt.title('Recall vs. Max Eigenvalues (Laplacian Matrix)')
plt.xlabel('Recall')
plt.ylabel('Real Max Eigenvalues')
plt.tight_layout()
plt.show()

In [ ]:
results['20']['query']

In [ ]:
results['0']['query']

# Compute average for more queries

In [ ]:
# function to save obj to pickle file
def save_obj(obj, filename):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(filename):
    ret_obj = None
    with open(filename, 'rb') as f:
        ret_obj = pickle.load(f)
    return ret_obj

In [ ]:
# In progress...

def get_eigens_from_results(sample_res, paper_id_pool, return_full_res=False):    
    recalls = []
    numbers_of_edges = []
    adjacency_eigenvalues = []
    laplacian_eigenvalues = []

    # storing the full results
    full_adjacency_eigenvalues_list = []
    full_laplacian_eigenvalues_list = []
    
    for num in range(len(sample_res)):
        swapped_res = random_paper_id_swap(sample_res, remove_id_from_list(paper_id_pool, sample_res), num, from_end=True)
        mg = MetadataGraph()
        for k in swapped_res:
            mg.update_one(g_data.data_dict[k])

        numbers_of_edges.append(mg.G.number_of_edges())
        
        adj_eigenvalues, _ = get_eigen(mg.get_adjacency_matrix())
        adjacency_eigenvalues.append(adj_eigenvalues.real.max())

        lap_eigenvalues, _ = get_eigen(mg.get_laplacian_matrix(normalized=True))
        laplacian_eigenvalues.append(lap_eigenvalues.real.max())
        # eigenvalues = eigenvalues[eigenvalues > 0]
        # real_min_eigen = eigenvalues.real.min()
        # laplacian_eigenvalues.append(real_min_eigen)
        recalls.append( (len(sample_res) - num) / len(sample_res) )

        if return_full_res:
            full_adjacency_eigenvalues_list.append(adj_eigenvalues)
            full_laplacian_eigenvalues_list.append(lap_eigenvalues)

    if return_full_res:
        return recalls, numbers_of_edges, adjacency_eigenvalues, laplacian_eigenvalues, (full_adjacency_eigenvalues_list, full_laplacian_eigenvalues_list)
    return recalls, numbers_of_edges, adjacency_eigenvalues, laplacian_eigenvalues

In [ ]:
G_data = pickle.load(open('../data/arxiv/graph.pickle', 'rb'))

# read json result file
results = json.load(open('../data/arxiv/arxiv_workloads/res/k100_gk20_pn50_n20.json', 'r'))
paper_id_pool = list(G_data.data_dict.keys())

In [ ]:
# sample_res = results['0']['arxiv_abstract']
recalls_list = []
numbers_of_edges = []
adjacency_eigenvalues = []
laplacian_eigenvalues = []

Full_eigenvalue_res = []

for index, res_k in tqdm(enumerate(results.keys()), total=len(results.keys())):
    sample_res = results[res_k]['arxiv_abstract']

    # recalls, edges, adj_eig, lap_eig = get_eigens_from_results(sample_res, paper_id_pool)
    recalls, edges, adj_eig, lap_eig, full_res = get_eigens_from_results(sample_res, paper_id_pool, return_full_res=True)
    recalls_list.append(recalls)
    numbers_of_edges.append(edges)
    adjacency_eigenvalues.append(adj_eig)
    laplacian_eigenvalues.append(lap_eig)

    Full_eigenvalue_res.append(full_res)

    if index==3:
        break

In [ ]:
# save Full_eigenvalue_res
save_obj(Full_eigenvalue_res, '../data/arxiv/arxiv_full_eigenvalue_res.pickle')

tmp = load_obj('../data/arxiv/tmp/arxiv_full_eigenvalue_res.pickle')
tmp_adj_res, tmp_lap_res = tmp[0]

In [ ]:
# compute avg 
recalls = np.array(recalls_list)
numbers_of_edges = np.array(numbers_of_edges)
adjacency_eigenvalues = np.array(adjacency_eigenvalues)
laplacian_eigenvalues = np.array(laplacian_eigenvalues)

avg_recalls = recalls.mean(axis=0)
avg_edges = numbers_of_edges.mean(axis=0)
avg_adj_eig = adjacency_eigenvalues.mean(axis=0)
avg_lap_eig = laplacian_eigenvalues.mean(axis=0)

In [ ]:
numbers_of_edges.shape

In [ ]:
plt.plot(avg_recalls, avg_adj_eig)
plt.title('Recall vs. Max Eigenvalues (adj Matrix)')
plt.xlabel('Recall')
plt.ylabel('Real Max Eigenvalues')
plt.tight_layout()
# plt.savefig("Laplacian_Eigen_k100_gk20_pn50_n20.png")
plt.show()

In [ ]:
plt.plot(avg_recalls, avg_lap_eig)
plt.title('Recall vs. Max Eigenvalues (Laplacian Matrix)')
plt.xlabel('Recall')
plt.ylabel('Real Max Eigenvalues')
plt.tight_layout()
# plt.savefig("Laplacian_Eigen_k100_gk20_pn50_n20.png")
plt.show()

# Recall vs. Egien runtime

Keep adding element to list, will Egienvalue be increasing. 

Note: Direct comparison with graph that have different numbers of node. Need to normalixation. 

In [ ]:
def get_eigens_from_results_runtime(sample_res):    
    number_of_elements = []
    numbers_of_edges = []
    numbers_of_nodes = []
    adjacency_eigenvalues = []
    laplacian_eigenvalues = []
    
    for num in range(len(sample_res)):
        num+=1
        # pprint(num)
        # only take a subset of the sample_res (num)
        partial_data_samples = sample_res[:num]
        number_of_elements.append(len(partial_data_samples))
        
        mg = MetadataGraph()
        for k in partial_data_samples:
            mg.update_one(g_data.data_dict[k])

        numbers_of_edges.append(mg.G.number_of_edges())
        numbers_of_nodes.append(mg.G.number_of_nodes())
        
        eigenvalues, _ = get_eigen(mg.get_adjacency_matrix())
        adjacency_eigenvalues.append(eigenvalues.real.max())

        eigenvalues, _ = get_eigen(mg.get_laplacian_matrix(normalized=True))
        laplacian_eigenvalues.append(eigenvalues.real.max())
        # recalls.append( (len(sample_res) - num) / len(sample_res) )
        
    return number_of_elements, numbers_of_nodes, numbers_of_edges, adjacency_eigenvalues, laplacian_eigenvalues

In [ ]:
def build_metadata_graph(id_list, old_graph_obj, step=None):
    if step==None:
        step = len(id_list)
        
    mg = MetadataGraph()
    for k in id_list[:step]:
        mg.update_one(old_graph_obj.data_dict[k])
    return mg

mg = build_metadata_graph(results['10']['arxiv_abstract'], g_data, 5)

# eigenvalues, _ = get_eigen(mg.get_laplacian_matrix())
# eigenvalues

In [ ]:
mg.get_adjacency_matrix().shape

In [ ]:
mg.get_laplacian_matrix()

In [ ]:
# len(mg.metadata_nodes)

# t = create_random_adjacency_matrix(10)
# eigenvalues, _ = get_eigen(t)
# sorted(eigenvalues, reverse=True)

In [ ]:
G_data = pickle.load(open('../data/arxiv/graph.pickle', 'rb'))
results = json.load(open('../data/arxiv/arxiv_workloads/res/k100_gk20_pn50_n20.json', 'r'))
paper_id_pool = list(G_data.data_dict.keys())

In [ ]:
tmp = results['0']['arxiv_abstract']
num_elements1, num_nodes1, edges1, adj_eig1, lap_eig1 = get_eigens_from_results_runtime(tmp)

In [ ]:
'''
Make 3 subplots in the same row:
- plot num_elements vs. lap_eig
- plot num_elements vs. adj_eig
- plot num_elements vs. edges
'''

fig, ax = plt.subplots(2, 2, figsize=(9, 5.5))

ax[0][0].plot(num_elements1, lap_eig1)
ax[0][0].set_title('Number of Elements vs. Max Eigenvalues\n(Laplacian Matrix)')
ax[0][0].set_xlabel('Number of Elements')
ax[0][0].set_ylabel('Real Max Eigenvalues')

ax[0][1].plot(num_elements1, adj_eig1)
ax[0][1].set_title('Number of Elements vs. Max Eigenvalues\n(Adjacency Matrix)')
ax[0][1].set_xlabel('Number of Elements')
ax[0][1].set_ylabel('Real Max Eigenvalues')

ax[1][0].plot(num_elements1, edges1)
ax[1][0].set_title('Number of Elements vs. Number of Edges')
ax[1][0].set_xlabel('Number of Elements')
ax[1][0].set_ylabel('Number of Edges')

ax[1][1].plot(num_elements1, num_nodes1)
ax[1][1].set_title('Number of Elements vs. Number of Nodes')
ax[1][1].set_xlabel('Number of Elements')
ax[1][1].set_ylabel('Number of Nodes')

plt.suptitle("Recall: 100%")
plt.tight_layout()
plt.show()

In [ ]:
random_dps = random.sample(paper_id_pool, 100)
num_elements, num_nodes, edges, adj_eig, lap_eig = get_eigens_from_results_runtime(random_dps)

fig, ax = plt.subplots(2, 2, figsize=(9, 5.5))

ax[0][0].plot(num_elements, lap_eig)
ax[0][0].set_title('Number of Elements vs. Max Eigenvalues\n(Laplacian Matrix)')
ax[0][0].set_xlabel('Number of Elements')
ax[0][0].set_ylabel('Real Max Eigenvalues')

ax[0][1].plot(num_elements, adj_eig)
ax[0][1].set_title('Number of Elements vs. Max Eigenvalues\n(Adjacency Matrix)')
ax[0][1].set_xlabel('Number of Elements')
ax[0][1].set_ylabel('Real Max Eigenvalues')

ax[1][0].plot(num_elements, edges)
ax[1][0].set_title('Number of Elements vs. Number of Edges')
ax[1][0].set_xlabel('Number of Elements')
ax[1][0].set_ylabel('Number of Edges')

ax[1][1].plot(num_elements, num_nodes)
ax[1][1].set_title('Number of Elements vs. Number of Nodes')
ax[1][1].set_xlabel('Number of Elements')
ax[1][1].set_ylabel('Number of Nodes')

plt.suptitle("Random Data Points")
plt.tight_layout()
plt.show()

In [ ]:
plt.plot(num_elements, lap_eig)   
plt.plot(num_elements1, lap_eig1)

## Avg workload adding elements plot

In [ ]:
# tmp = results['0']['arxiv_abstract']
# num_elements1, num_nodes1, edges1, adj_eig1, lap_eig1 = get_eigens_from_results_runtime(tmp)

In [ ]:
res_key_key = 'arxiv_abstract'

num_elements_list = []
num_nodes_list = []
num_edges_list = []
adj_eig_list = []
lap_eig_list = []

rd_num_elements_list = []
rd_num_nodes_list = []
rd_num_edges_list = []
rd_adj_eig_list = []
rd_lap_eig_list = []

# loop through dict keys
counter = 1
for k in tqdm(results):
    num_elements1, num_nodes1, edges1, adj_eig1, lap_eig1 = get_eigens_from_results_runtime(results[k][res_key_key])
    num_elements_list.append(num_elements1)
    num_nodes_list.append(num_nodes1)
    num_edges_list.append(edges1)
    adj_eig_list.append(adj_eig1)
    lap_eig_list.append(lap_eig1)

    random_dps = random.sample(paper_id_pool, 100)
    rd_num_elements, rd_num_nodes, rd_edges, rd_adj_eig, rd_lap_eig = get_eigens_from_results_runtime(random_dps)
    rd_num_elements_list.append(rd_num_elements)
    rd_num_nodes_list.append(rd_num_nodes)
    rd_num_edges_list.append(rd_edges)
    rd_adj_eig_list.append(rd_adj_eig)
    rd_lap_eig_list.append(rd_lap_eig)
    
    # if counter==3: break
    counter+=1

In [ ]:
# workload avg
num_elements_list = np.array(num_elements_list)
num_nodes_list = np.array(num_nodes_list)
num_edges_list = np.array(numbers_of_edges)
adj_eig_list = np.array(adj_eig_list)
lap_eig_list = np.array(lap_eig_list)

# compute mean
avg_num_elements = num_elements_list.mean(axis=0)
avg_num_nodes = num_nodes_list.mean(axis=0)
avg_num_edges = num_edges_list.mean(axis=0)
avg_adj_eig = adj_eig_list.mean(axis=0)
avg_lap_eig = lap_eig_list.mean(axis=0)


# do the same for rd
rd_num_elements_list = np.array(rd_num_elements_list)
rd_num_nodes_list = np.array(rd_num_nodes_list)
rd_num_edges_list = np.array(rd_num_edges_list)
rd_adj_eig_list = np.array(rd_adj_eig_list)
rd_lap_eig_list = np.array(rd_lap_eig_list)

rd_avg_num_elements = rd_num_elements_list.mean(axis=0)
rd_avg_num_nodes = rd_num_nodes_list.mean(axis=0)
rd_avg_num_edges = rd_num_edges_list.mean(axis=0)
rd_avg_adj_eig = rd_adj_eig_list.mean(axis=0)
rd_avg_lap_eig = rd_lap_eig_list.mean(axis=0)

In [ ]:
# plot avg with min max shade range, 2 plots: adj and lap eig
fig, ax = plt.subplots(1, 2, figsize=(11, 3.5))
ax[0].plot(avg_num_elements, avg_adj_eig, label='GT data points')
# ax[0].fill_between(avg_num_elements, adj_eig_list.min(axis=0), adj_eig_list.max(axis=0), alpha=0.3)

ax[1].plot(avg_num_elements, avg_lap_eig, label='GT data points')
ax[1].fill_between(avg_num_elements, lap_eig_list.min(axis=0), lap_eig_list.max(axis=0), alpha=0.3)

# add rd_line
ax[0].plot(rd_num_elements, rd_avg_adj_eig, label='Random data points', color='red', linestyle='--')
ax[1].plot(rd_num_elements, rd_avg_lap_eig, label='Random data points', color='red', linestyle='--')

ax[0].set_title('Eigenvalue (Adjacency)')
ax[0].set_xlabel('Number of Elements')
ax[0].set_ylabel('Real Max Eigenvalues')
ax[0].legend(loc='lower right')

ax[1].set_title('Eigenvalues (Laplacian)')
ax[1].set_xlabel('Number of Elements')
ax[1].set_ylabel('Real Max Eigenvalues')
ax[1].legend(loc='lower right')

plt.tight_layout()
# plt.savefig("avg_eigen_adding_elements2.png")
plt.show()